In [1]:
from typing import List

import pandas as pd
from pathlib import Path
import os
import sys
from dotenv import load_dotenv, dotenv_values
from pydriller import Repository
import json
from multiprocessing import Pool
from itertools import chain

In [2]:
load_dotenv()
ENV = dotenv_values(".env")
DATA_DIR = Path(ENV["DATA_DIR"])
DATA_DIR

PosixPath('/home/mm/w/jpatch/data')

In [3]:
df = pd.read_csv(DATA_DIR / 'proc' / 'changes.tsv', delimiter='\t')
df

,cveId,ghsaId,packageName,vulnIntroduced,latestVulnVersion,fixedVersion,fixCommitRepo,fixCommitHash,jarBefore,jarAfter,srcBefore,srcAfter,classFileBefore,classFileAfter,changedClass,changedMethods,packageIdBefore,packageIdAfter,diffParsedJson,nloc
0,CVE-2013-7285,GHSA-f554-x222-wgf7,com.thoughtworks.xstream:xstream,0,1.4.5,1.4.6,https://github.com/x-stream/xstream,6344867dce6767af7d0fe34fb393271a6456672d,proc/changes/CVE-2013-7285/6344867dce6767af7d0...,proc/changes/CVE-2013-7285/6344867dce6767af7d0...,proc/changes/CVE-2013-7285/6344867dce6767af7d0...,proc/changes/CVE-2013-7285/6344867dce6767af7d0...,proc/changes/CVE-2013-7285/6344867dce6767af7d0...,proc/changes/CVE-2013-7285/6344867dce6767af7d0...,XStream,"[XStream::allowTypes, XStream::denyPermission,...",com.thoughtworks.xstream,com.thoughtworks.xstream,"{""added"": [[3, "" * Copyright (C) 2006, 2007, 2...",1032
1,CVE-2019-17513,GHSA-mvqp-q37c-wf9j,io.ratpack:ratpack-core,0,1.7.4,1.7.5,https://github.com/ratpack/ratpack,efb910d38a96494256f36675ef0e5061097dd77d,proc/changes/CVE-2019-17513/efb910d38a96494256...,proc/changes/CVE-2019-17513/efb910d38a96494256...,proc/changes/CVE-2019-17513/efb910d38a96494256...,proc/changes/CVE-2019-17513/efb910d38a96494256...,proc/changes/CVE-2019-17513/efb910d38a96494256...,proc/changes/CVE-2019-17513/efb910d38a96494256...,NettyHandlerAdapter,[NettyHandlerAdapter::newRequest],ratpack.server.internal,ratpack.server.internal,"{""added"": [[159, "" HttpHeaders nettyHeaders...",219
2,CVE-2019-16771,GHSA-24r8-fm9r-cpj2,com.linecorp.armeria:armeria,0,0.96.0,0.97.0,https://github.com/line/armeria,b597f7a865a527a84ee3d6937075cfbb4470ed20,proc/changes/CVE-2019-16771/b597f7a865a527a84e...,proc/changes/CVE-2019-16771/b597f7a865a527a84e...,proc/changes/CVE-2019-16771/b597f7a865a527a84e...,proc/changes/CVE-2019-16771/b597f7a865a527a84e...,proc/changes/CVE-2019-16771/b597f7a865a527a84e...,proc/changes/CVE-2019-16771/b597f7a865a527a84e...,HttpHeaderNames,"[HttpHeaderNames::create, HttpHeaderNames::mal...",com.linecorp.armeria.common,com.linecorp.armeria.common,"{""added"": [[35, ""import java.util.BitSet;""], [...",226
3,CVE-2019-16771,GHSA-24r8-fm9r-cpj2,com.linecorp.armeria:armeria,0,0.96.0,0.97.0,https://github.com/line/armeria,b597f7a865a527a84ee3d6937075cfbb4470ed20,proc/changes/CVE-2019-16771/b597f7a865a527a84e...,proc/changes/CVE-2019-16771/b597f7a865a527a84e...,proc/changes/CVE-2019-16771/b597f7a865a527a84e...,proc/changes/CVE-2019-16771/b597f7a865a527a84e...,proc/changes/CVE-2019-16771/b597f7a865a527a84e...,proc/changes/CVE-2019-16771/b597f7a865a527a84e...,HttpHeadersBase,"[HttpHeadersBase::set, HttpHeadersBase::setObj...",com.linecorp.armeria.common,com.linecorp.armeria.common,"{""added"": [[43, ""import java.util.BitSet;""], [...",977
4,CVE-2019-16771,GHSA-24r8-fm9r-cpj2,com.linecorp.armeria:armeria,0,0.96.0,0.97.0,https://github.com/line/armeria,b597f7a865a527a84ee3d6937075cfbb4470ed20,proc/changes/CVE-2019-16771/b597f7a865a527a84e...,proc/changes/CVE-2019-16771/b597f7a865a527a84e...,proc/changes/CVE-2019-16771/b597f7a865a527a84e...,proc/changes/CVE-2019-16771/b597f7a865a527a84e...,proc/changes/CVE-2019-16771/b597f7a865a527a84e...,proc/changes/CVE-2019-16771/b597f7a865a527a84e...,ArmeriaHttpUtil,[ArmeriaHttpUtil::toLowercaseMap],com.linecorp.armeria.internal,com.linecorp.armeria.internal,"{""added"": [[676, "" final AsciiStrin...",638
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,CVE-2020-13692,GHSA-88cc-g835-76rp,org.postgresql:postgresql,0,42.2.12.jre7,42.2.13,https://github.com/pgjdbc/pgjdbc,14b62aca4764d496813f55a43d050b017e01eb65,proc/changes/CVE-2020-13692/14b62aca4764d49681...,proc/changes/CVE-2020-13692/14b62aca4764d49681...,proc/changes/CVE-2020-13692/14b62aca4764d49681...,proc/changes/CVE-2020-13692/14b62aca4764d49681...,proc/changes/CVE-2020-13692/14b62aca4764d49681...,proc/changes/CVE-2020-13692/14b62aca4764d49681...,PgConnection,"[PgConnection::PgConnection, PgConnection::get...",org.postgre

In [15]:
df.apply(lambda x: x['cveId'] + '#' + x['packageIdAfter'] + ":" + x['changedClass'], axis=1).unique().__len__()

209

In [4]:
len(df['fixCommitRepo'].unique())

77

In [8]:
len(set([Path(x).name for x in df['jarAfter'].unique()]))

104

In [11]:
df['cveId'].unique().__len__(), df['ghsaId'].unique().__len__()

(108, 108)

In [14]:
df.apply(lambda x: x['packageIdAfter'] + ":" + x['changedClass'], axis=1).unique().__len__()

186